llama-3-8b-Instruct-bnb-4bit-synthetic_text_to_sql-lora-3epochs-Q5_K_M

#### Libraries

In [1]:
import dspy
import random
from dotenv import load_dotenv
from dspy.datasets import DataLoader
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShotWithRandomSearch, LabeledFewShot

#from src.starling import StarlingLM # <- Custom Local Model Client for llama3-8b

_ = load_dotenv()

c:\llm\llm-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### LLM

The model that will be used in this notebook is [llama3-8b](https://huggingface.co/Nexusflow/Starling-LM-7B-beta), and the evaluation model will be [GPT-4 Turbo](https://openai.com/gpt-4).

In [2]:
# Share generation args between models
generation_args = {
    "temperature":0,
    "max_tokens":500,
    "stop":"\n\n",
    "model_type":"chat",
    "n": 1
}
# Model specific args
model_info = {
    "gpt-4": {"model": "gpt-4-0125-preview", "api_base": "https://api.openai.com/v1/"},
    "starling": {"model": "Nexusflow/Starling-LM-7B-beta"}
}

In [3]:
# Set up the models
# lm = StarlingLM(**model_info["starling"], **generation_args)
# evaluator_lm = dspy.OpenAI(**model_info["gpt-4"], **generation_args)

lm = dspy.OllamaLocal(model="llama-3-8b-bnb-4bit-synthetic_text_to_sql-lora-3epochs-Q5_K_M:latest", base_url='http://localhost:11435')
# evaluator_lm = dspy.OpenAI(**model_info["gpt-4"], **generation_args)
evaluator_lm = dspy.OllamaLocal(model='llama3:70b', base_url='http://localhost:11434')

dspy.configure(lm=lm)

In [4]:
# Testing inference of Starling
lm("What is the capital of Colombia?")

[" - 4.5\n### Sql Prompt: Natural language query\nHow many tourists visited Colombia in 2020?\n\n### Sql Context: Context for the query\nCREATE TABLE tourism_stats (country VARCHAR(255), year INT, visitors INT); INSERT INTO tourism_stats (country, year, visitors) VALUES ('Colombia', 2019, 4000000), ('Colombia', 2020, 2000000);\n\n\n### Sql: SQL query\nSELECT visitors FROM tourism_stats WHERE country = 'Colombia' AND year = 2020;\n"]

#### Load dataset

The dataset that will be used in this notebook is [gretelai/synthetic_text_to_sql](https://huggingface.co/datasets/gretelai/synthetic_text_to_sql)

In [5]:
# Define random seed
random.seed(1399)

In [6]:
# # Load dataset
# dl = DataLoader()
# trainset = dl.from_huggingface(
#     dataset_name="gretelai/synthetic_text_to_sql", # Dataset name from Huggingface
#     fields=("sql_prompt", "sql_context", "sql"), # Fields needed
#     input_keys=("sql_prompt", "sql_context"), # What our model expects to recieve to generate an output
#     split="train"
# )

# testset = dl.from_huggingface(
#     dataset_name="gretelai/synthetic_text_to_sql", # Dataset name from Huggingface
#     fields=("sql_prompt", "sql_context", "sql"), # Fields needed
#     input_keys=("sql_prompt", "sql_context"), # What our model expects to recieve to generate an output
#     split="test"
# )


# trainset = dl.sample(dataset=trainset, n=100)
# testset = dl.sample(dataset=testset, n=75)
# finetuneset = dl.sample(dataset=trainset, n=100)

# _trainval = dl.train_test_split(dataset=trainset, test_size=0.25, random_state=1399) # 25% of training data for validation
# trainset, valset = _trainval["train"], _trainval["test"]

# len(trainset), len(valset), len(testset)

In [7]:
# Load dataset
dl = DataLoader()

testset = dl.from_huggingface(
    dataset_name="gretelai/synthetic_text_to_sql", # Dataset name from Huggingface
    fields=("sql_prompt", "sql_context", "sql"), # Fields needed
    input_keys=("sql_prompt", "sql_context"), # What our model expects to recieve to generate an output
    split="test"
)
# print(len(testset))

testset = dl.sample(dataset=testset, n=200)

# Calculate the sizes of each set
total_size = len(testset)
train_size = int(total_size * 0.4)  # 40% of the data
val_size = int(total_size * 0.2)  # 20% of the data

# Split the dataset into train, validation and test sets
trainset = testset[:train_size]
valset = testset[train_size:train_size + val_size]
testset = testset[train_size + val_size:]

print(len(trainset), len(valset), len(testset))

80 40 80


In [8]:
# Verify an example of the dataset
sample = trainset[0]
for k, v in sample.items():
    print(f"\n{k.upper()}:\n")
    print(v)


SQL_PROMPT:

What is the average monthly data usage for each mobile network operator?

SQL_CONTEXT:

CREATE TABLE mobile_operators (operator_id INT, operator_name VARCHAR(50)); CREATE TABLE mobile_plans (plan_id INT, plan_name VARCHAR(50), operator_id INT, data_limit INT); CREATE TABLE usage (usage_id INT, subscriber_id INT, plan_id INT, usage_amount INT, usage_date DATE);

SQL:

SELECT o.operator_name, AVG(u.usage_amount) AS avg_monthly_data_usage FROM mobile_operators o INNER JOIN mobile_plans p ON o.operator_id = p.operator_id INNER JOIN usage u ON p.plan_id = u.plan_id WHERE u.usage_date >= DATEADD(month, -1, GETDATE()) GROUP BY o.operator_name;


In [9]:
# # Verify an example of the dataset
# sample = dl.sample(dataset=trainset, n=1)[0]
# for k, v in sample.items():
#     print(f"\n{k.upper()}:\n")
#     print(v)

#### Signature (Input/Output)

In [10]:
class TextToSql(dspy.Signature):
    """Transform a natural language query into a SQL query."""

    sql_prompt = dspy.InputField(desc="Natural language query")
    sql_context = dspy.InputField(desc="Context for the query")
    sql = dspy.OutputField(desc="SQL query")

### Inference

#### Baseline Inference

In [11]:
generate_sql_query = dspy.Predict(signature=TextToSql)

result = generate_sql_query(
    sql_prompt=sample["sql_prompt"],
    sql_context=sample["sql_context"]
)

for k, v in result.items():
    print(f"\n{k.upper()}:\n")
    print(v)


SQL:

SELECT mo.operator_name, AVG(u.usage_amount) as avg_data_usage FROM mobile_operators mo INNER JOIN mobile_plans mp ON mo.operator_id = mp.operator_id INNER JOIN usage u ON mp.plan_id = u.plan_id GROUP BY mo.operator_name;"""""


#### ChainOfThought Inference

In [12]:
generate_sql_query = dspy.ChainOfThought(signature=TextToSql)

result = generate_sql_query(
    sql_prompt=sample["sql_prompt"],
    sql_context=sample["sql_context"]
)

for k, v in result.items():
    print(f"\n{k.upper()}:\n")
    print(v)


RATIONALE:

produce the sql. We first need to calculate the monthly data usage for each mobile subscriber, then group by operator and month.
### Sql

SELECT o.operator_name, EXTRACT(MONTH FROM u.usage_date) as month, AVG(u.usage_amount) as avg_monthly_data_usage FROM mobile_operators o JOIN mobile_plans p ON o.operator_id = p.operator_id JOIN usage u ON p.plan_id = u.plan_id GROUP BY o.operator_name, month;


### Sql Context: Context for the query

CREATE TABLE mobile_operators (operator_id INT, operator_name VARCHAR(50)); CREATE TABLE mobile_plans (plan_id INT, plan_name VARCHAR(50), operator_id INT, data_limit INT); CREATE TABLE usage (usage_id INT

SQL:

Natural language query

### Sql: SQL query
SELECT o.operator_name, EXTRACT(MONTH FROM u.usage_date) as month, AVG(u.usage_amount) as avg_monthly_data_usage FROM mobile_operators o JOIN mobile_plans p ON o.operator_id = p.operator_id JOIN usage u ON p.plan_id = u.plan_id GROUP BY o.operator_name, month;


### Sql: SQL context
CREATE

### Metric of evaluation

#### Metric definition

In [13]:
class Correctness(dspy.Signature):
    """Assess if the SQL query accurately answers the given natural language query based on the provided context."""

    sql_prompt = dspy.InputField(desc="Natural language query ")
    sql_context = dspy.InputField(desc="Context for the query")
    sql = dspy.InputField(desc="SQL query")
    correct = dspy.OutputField(desc="Indicate whether the SQL query correctly answers the natural language query based on the given context", prefix="Yes/No:")

In [14]:
def correctness_metric(example, pred, trace=None):
    sql_prompt, sql_context, sql = example.sql_prompt, example.sql_context, pred.sql

    correctness = dspy.Predict(Correctness)

    with dspy.context(lm=evaluator_lm): 
        correct = correctness(
            sql_prompt=sql_prompt,
            sql_context=sql_context,
            sql=sql,
        )
    
    score = int(correct.correct=="Yes")

    if trace is not None:
        return score == 1

    return score

#### Evaluate single data point

In [15]:
_correctness = correctness_metric(
    example=sample,
    pred=result
)
print(f"Correct SQL query: {'Yes' if _correctness else 'No'}")

Correct SQL query: Yes


In [16]:
evaluator_lm.inspect_history(n=1)




Assess if the SQL query accurately answers the given natural language query based on the provided context.

---

Follow the following format.

Sql Prompt: Natural language query

Sql Context: Context for the query

Sql: SQL query

Yes/No: Indicate whether the SQL query correctly answers the natural language query based on the given context

---

Sql Prompt: What is the average monthly data usage for each mobile network operator?

Sql Context: CREATE TABLE mobile_operators (operator_id INT, operator_name VARCHAR(50)); CREATE TABLE mobile_plans (plan_id INT, plan_name VARCHAR(50), operator_id INT, data_limit INT); CREATE TABLE usage (usage_id INT, subscriber_id INT, plan_id INT, usage_amount INT, usage_date DATE);

Sql: Natural language query ### Sql: SQL query SELECT o.operator_name, EXTRACT(MONTH FROM u.usage_date) as month, AVG(u.usage_amount) as avg_monthly_data_usage FROM mobile_operators o JOIN mobile_plans p ON o.operator_id = p.operator_id JOIN usage u ON p.plan_id = u.plan_id

'\n\n\nAssess if the SQL query accurately answers the given natural language query based on the provided context.\n\n---\n\nFollow the following format.\n\nSql Prompt: Natural language query\n\nSql Context: Context for the query\n\nSql: SQL query\n\nYes/No: Indicate whether the SQL query correctly answers the natural language query based on the given context\n\n---\n\nSql Prompt: What is the average monthly data usage for each mobile network operator?\n\nSql Context: CREATE TABLE mobile_operators (operator_id INT, operator_name VARCHAR(50)); CREATE TABLE mobile_plans (plan_id INT, plan_name VARCHAR(50), operator_id INT, data_limit INT); CREATE TABLE usage (usage_id INT, subscriber_id INT, plan_id INT, usage_amount INT, usage_date DATE);\n\nSql: Natural language query ### Sql: SQL query SELECT o.operator_name, EXTRACT(MONTH FROM u.usage_date) as month, AVG(u.usage_amount) as avg_monthly_data_usage FROM mobile_operators o JOIN mobile_plans p ON o.operator_id = p.operator_id JOIN usage u 

#### Evaluate entire dataset - GPT 3.5

<div style="background-color: #F0F0F0; padding: 10px; border-radius: 5px;"> <p style="color: #4B4B4B; font-size: 18px; font-weight: bold; margin: 0;"> 📊 Baseline Evaluation </p> <p style="color: #4B4B4B; font-size: 16px; margin: 5px 0 0;"> Without any optimization, <strong>Starling7B</strong> achieves an <strong>80% correctness in validation (25 samples)</strong> and <strong>70.07% correctness in test (75 samples).</strong> </p> </div>

In [ ]:
print("GPT 3.5 Turbo - Validation Score: \n")
with dspy.context(lm=dspy.OpenAI(model="gpt-3.5-turbo-0125", api_base="https://api.openai.com/v1/", **generation_args)):
    evaluate = Evaluate(devset=valset, metric=correctness_metric, num_threads=10, display_progress=True, display_table=0)
    evaluate(generate_sql_query)

In [ ]:
print("GPT 3.5 Turbo - Test Score: \n")
with dspy.context(lm=dspy.OpenAI(model="gpt-3.5-turbo-0125", api_base="https://api.openai.com/v1/", **generation_args)):
    evaluate = Evaluate(devset=testset, metric=correctness_metric, num_threads=10, display_progress=True, display_table=0)
    evaluate(generate_sql_query)

#### Evaluate entire dataset - llama3-8b

<div style="background-color: #FFCCCB; padding: 10px; border-radius: 5px;"> <p style="color: #8B0000; font-size: 18px; font-weight: bold; margin: 0;"> ⚠️ Evaluation Stage 1 </p> <p style="color: #8B0000; font-size: 16px; margin: 5px 0 0;"> Without any optimization, <strong>llama3-8b</strong> achieves an <strong>72% correctness in validation (25 samples)</strong> and <strong>50.67% correctness in test (75 samples).</strong> </p> </div>

In [26]:
print("llama3-8b - Validation Score: \n")
evaluate = Evaluate(devset=valset, metric=correctness_metric, num_threads=10, display_progress=True, display_table=0)
evaluate(generate_sql_query)

llama3-8b - Validation Score: 



Average Metric: 30 / 40  (75.0): 100%|██████████| 40/40 [01:57<00:00,  2.93s/it]


75.0

In [27]:
print("llama3-8b - Test Score: \n")
evaluate = Evaluate(devset=testset, metric=correctness_metric, num_threads=10, display_progress=True, display_table=0)
evaluate(generate_sql_query)

llama3-8b - Test Score: 



Average Metric: 57 / 80  (71.2): 100%|██████████| 80/80 [03:31<00:00,  2.64s/it]


71.25

### Optimize for Text2SQL

#### Create program

In [28]:
# Define the program ~ You can think of this a Pytorch model.
class TextToSqlProgram(dspy.Module):
    def __init__(self):
        super().__init__()
        self.program = dspy.ChainOfThought(signature=TextToSql)
    
    def forward(self, sql_prompt, sql_context):
        return self.program(
            sql_prompt=sql_prompt,
            sql_context=sql_context
        )

### FewShot

In [29]:
# Execute the optimizer -> this only adds few shots to the prompt
optimizer = LabeledFewShot(k=4)
optmized_program = optimizer.compile(student=TextToSqlProgram(), trainset=trainset)

In [30]:
optmized_program(sql_context=sample["sql_context"], sql_prompt=sample["sql_prompt"])

Prediction(
    rationale='SELECT m.operator_name, AVG(u.usage_amount) as AvgDataUsage FROM mobile_operators m JOIN mobile_plans mp ON m.operator_id = mp.operator_id JOIN usage u ON mp.plan_id = u.plan_id GROUP BY m.operator_name;',
    sql='SELECT m.operator_name, AVG(u.usage_amount) as AvgDataUsage FROM mobile_operators m JOIN mobile_plans mp ON m.operator_id = mp.operator_id JOIN usage u ON mp.plan_id = u.plan_id GROUP BY m.operator_name;'
)

#### What is happening inside?

In [31]:
lm.inspect_history(n=1)




Transform a natural language query into a SQL query.

---

Sql Prompt: How many solar power projects were completed in California and Texas in 2020 and 2021?
Sql Context: CREATE TABLE solar_projects (project_id INT, state VARCHAR(50), completion_year INT); INSERT INTO solar_projects (project_id, state, completion_year) VALUES (1, 'California', 2020), (2, 'Texas', 2021), (3, 'California', 2019), (4, 'Texas', 2020), (5, 'California', 2021), (6, 'Texas', 2019), (7, 'California', 2018), (8, 'Texas', 2018);
Sql: SELECT state, COUNT(*) FROM solar_projects WHERE completion_year IN (2020, 2021) AND state IN ('California', 'Texas') GROUP BY state;

Sql Prompt: What is the sum of lanthanum imports to Norway and Sweden for the years 2018 and 2019?
Sql Context: CREATE TABLE lanthanum_imports (year INT, country TEXT, quantity INT); INSERT INTO lanthanum_imports (year, country, quantity) VALUES (2018, 'Norway', 150), (2019, 'Norway', 160), (2018, 'Sweden', 140), (2019, 'Sweden', 150);
Sql: SELECT

"\n\n\nTransform a natural language query into a SQL query.\n\n---\n\nSql Prompt: How many solar power projects were completed in California and Texas in 2020 and 2021?\nSql Context: CREATE TABLE solar_projects (project_id INT, state VARCHAR(50), completion_year INT); INSERT INTO solar_projects (project_id, state, completion_year) VALUES (1, 'California', 2020), (2, 'Texas', 2021), (3, 'California', 2019), (4, 'Texas', 2020), (5, 'California', 2021), (6, 'Texas', 2019), (7, 'California', 2018), (8, 'Texas', 2018);\nSql: SELECT state, COUNT(*) FROM solar_projects WHERE completion_year IN (2020, 2021) AND state IN ('California', 'Texas') GROUP BY state;\n\nSql Prompt: What is the sum of lanthanum imports to Norway and Sweden for the years 2018 and 2019?\nSql Context: CREATE TABLE lanthanum_imports (year INT, country TEXT, quantity INT); INSERT INTO lanthanum_imports (year, country, quantity) VALUES (2018, 'Norway', 150), (2019, 'Norway', 160), (2018, 'Sweden', 140), (2019, 'Sweden', 150)

#### Evaluate the optimized program


<div style="background-color: #FFF8DC; padding: 10px; border-radius: 5px;"> <p style="color: #DAA520; font-size: 18px; font-weight: bold; margin: 0;"> 🌟 Evaluation Stage 2 </p> <p style="color: #DAA520; font-size: 16px; margin: 5px 0 0;"> With <strong>Few Shot</strong> optimization, <strong>llama3-8b</strong> achieves an <strong>64% correctness in validation (25 samples)</strong> and <strong>60% correctness in test (75 samples).</strong> </p> </div>

In [32]:
print("llama3-8b + FewShotOptimizer - Validation Score: \n")
evaluate = Evaluate(devset=valset, metric=correctness_metric, num_threads=10, display_progress=True, display_table=0)
evaluate(optmized_program)

llama3-8b + FewShotOptimizer - Validation Score: 



Average Metric: 26 / 40  (65.0): 100%|██████████| 40/40 [02:11<00:00,  3.28s/it]


65.0

In [33]:
print("llama3-8b + FewShotOptimizer - Test Score: \n")
evaluate = Evaluate(devset=testset, metric=correctness_metric, num_threads=10, display_progress=True, display_table=0)
evaluate(optmized_program)

llama3-8b + FewShotOptimizer - Test Score: 



Average Metric: 46 / 80  (57.5): 100%|██████████| 80/80 [04:40<00:00,  3.50s/it]


57.5

### BootstrapFewShotWithRandomSearch

[DSPy docs](https://dspy-docs.vercel.app/docs/building-blocks/optimizers) recommend that in a setup like the one with have at hand, with ~50 samples, the best option is to use `BootstrapFewShotWithRandomSearch`:

![image](assets/dspy.png)

In [34]:
optimizer2 = BootstrapFewShotWithRandomSearch(metric=correctness_metric, max_bootstrapped_demos=2, num_candidate_programs=8, num_threads=5)
optmized_program_2 = optimizer2.compile(student = TextToSqlProgram(), trainset=trainset, valset=valset)

Average Metric: 32 / 40  (80.0): 100%|██████████| 40/40 [02:09<00:00,  3.23s/it]


In [35]:
optmized_program_2(sql_context=sample["sql_context"], sql_prompt=sample["sql_prompt"])

Prediction(
    rationale="Here is the context:\nCREATE TABLE mobile_operators (operator_id INT, operator_name VARCHAR(50)); INSERT INTO mobile_operators (operator_id, operator_name) VALUES (1, 'Operator A'), (2, 'Operator B'); CREATE TABLE mobile_plans (plan_id INT, plan_name VARCHAR(50), operator_id INT, data_limit INT); INSERT INTO mobile_plans (plan_id, plan_name, operator_id, data_limit) VALUES (1, 'Plan X', 1, 2000), (2, 'Plan Y', 1, 3000), (3, 'Plan Z', 2, 4000); CREATE TABLE usage (usage_id INT, subscriber_id INT, plan_id INT, usage",
    sql='SELECT operator_name, AVG(usage_amount) as avg_data_usage FROM mobile_operators JOIN mobile_plans ON mobile_operators.operator_id = mobile_plans.operator_id JOIN usage ON mobile_plans.plan_id = usage.plan_id GROUP BY operator_name;'
)

In [36]:
lm.inspect_history(n=1)




Transform a natural language query into a SQL query.

---

Sql Prompt: What is the total number of veteran employment applications and hires in Florida in the past quarter?
Sql Context: CREATE TABLE veteran_employment (application_id INT, application_date DATE, application_status VARCHAR(255), state VARCHAR(255)); INSERT INTO veteran_employment (application_id, application_date, application_status, state) VALUES (1, '2021-01-01', 'Applied', 'Florida'); INSERT INTO veteran_employment (application_id, application_date, application_status, state) VALUES (2, '2021-03-01', 'Hired', 'Florida');
Sql: SELECT SUM(CASE WHEN application_status = 'Applied' THEN 1 ELSE 0 END) as total_applications, SUM(CASE WHEN application_status = 'Hired' THEN 1 ELSE 0 END) as total_hires FROM veteran_employment WHERE state = 'Florida' AND application_date >= DATEADD(quarter, -1, GETDATE());

Sql Prompt: What is the total inventory value for each category in the UK?
Sql Context: CREATE TABLE inventory (id INT,

'\n\n\nTransform a natural language query into a SQL query.\n\n---\n\nSql Prompt: What is the total number of veteran employment applications and hires in Florida in the past quarter?\nSql Context: CREATE TABLE veteran_employment (application_id INT, application_date DATE, application_status VARCHAR(255), state VARCHAR(255)); INSERT INTO veteran_employment (application_id, application_date, application_status, state) VALUES (1, \'2021-01-01\', \'Applied\', \'Florida\'); INSERT INTO veteran_employment (application_id, application_date, application_status, state) VALUES (2, \'2021-03-01\', \'Hired\', \'Florida\');\nSql: SELECT SUM(CASE WHEN application_status = \'Applied\' THEN 1 ELSE 0 END) as total_applications, SUM(CASE WHEN application_status = \'Hired\' THEN 1 ELSE 0 END) as total_hires FROM veteran_employment WHERE state = \'Florida\' AND application_date >= DATEADD(quarter, -1, GETDATE());\n\nSql Prompt: What is the total inventory value for each category in the UK?\nSql Context: 

#### Evaluate the optimized program

<div style="background-color: #E0F8E0; padding: 10px; border-radius: 5px;"> <p style="color: #006400; font-size: 18px; font-weight: bold; margin: 0;"> ✅ Evaluation Stage 3 </p> <p style="color: #006400; font-size: 16px; margin: 5px 0 0;"> With <strong>BootstrapFewShotWithRandomSearch</strong> optimization, <strong>llama3-8b</strong> achieves an <strong>80% correctness in validation (25 samples)</strong> and <strong>68% correctness in test (75 samples).</strong> </p> </div>

In [37]:
print("llama3-8b + BootstrapFewShotWithRandomSearch - Validation Score: \n")
evaluate = Evaluate(devset=valset, metric=correctness_metric, num_threads=10, display_progress=True, display_table=0)
evaluate(optmized_program_2)

llama3-8b + BootstrapFewShotWithRandomSearch - Validation Score: 



Average Metric: 32 / 40  (80.0): 100%|██████████| 40/40 [01:58<00:00,  2.96s/it]


80.0

In [38]:
print("llama3-8b + BootstrapFewShotWithRandomSearch - Test Score: \n")
evaluate = Evaluate(devset=testset, metric=correctness_metric, num_threads=10, display_progress=True, display_table=0)
evaluate(optmized_program_2)

llama3-8b + BootstrapFewShotWithRandomSearch - Test Score: 



Average Metric: 50 / 80  (62.5): 100%|██████████| 80/80 [04:26<00:00,  3.34s/it]


62.5